# Notes

**Focus on**:
- Quality of data
- Log your data and calls when using data scraping
- Creativity
- Visualization (explanatory figures), simple is better
- Be critical of your data collection and generating process
    - Bias
    - Missing data
        - Ignore
        - Collect new data
        - Remove or replace missing data
    - Internal and external validity
    - Data collection type (random, survey, big data, other)
- Less focus on the analytical section and more on the collection and presentation

### Reflect on the ethical aspect
- Do yourespectprivacy? 
- Can single individualsbeidentified? 
- What are the potential consequences?
- Are there ethical considerations?
    - With respect to individuals? 
    - With respectto firmsor organizations?
- Consider the GDPR:
    - Is it anonymous? 
    - Personal data or statistics?
    - Any change of re-identification?

### Project ideas

- Factor based investing (use google or yahoo finance API)
- Trump tweet correlation with stock market
- Don't use LinkedIn
- 


### Logging

- Log your calls, use it to determine success ratio
    - Where did the call fail? Rewrite code.
    - Don't be greedy. time.sleep(0.5) between each call.
- Visualize the log (lecture 10)

We start by importing our data source to Python. The file *tweets.json* is created from [the Trump twitter archive](http://www.trumptwitterarchive.com/archive). We have selected all tweets from the 20th of January 2017 (assumed office) to 21st of August 2019.

In [ ]:
# CODE BIN:
# # Loading dataset from file using a relative path
# with codecs.open('tweets.json', 'r', 'utf-8-sig') as json_file:  
#     data = json.load(json_file)
#     df=pd.DataFrame(data) # converting to a pandas data frame
    

In [96]:
# Importing packages
import pandas as pd
import json, codecs
import scraping_class, time

logfile = 'my_log'## name your log file.
connector = scraping_class.Connector(logfile)
data = []
# Fetching data
for i in range(2017,2020):
    url = 'http://www.trumptwitterarchive.com/data/realdonaldtrump/'+str(i)+'.json'
    r, call_id = connector.get(url, 'Tweets')
    json_file = r.json() 
    data += json_file[::-1] # invert list
    time.sleep(0.5) # set sleep timer to prevent unintentional DOS attacks

# Creating and manipulating dataframe    
df = pd.DataFrame(data)
keywords = ['China', 'tariff','Tariff'] # list of relevant keywords
df['keyword'] = df['text'].str.contains('|'.join(keywords))
df_filter = df.query("is_retweet == False & keyword == True")
display(df_filter.head())

# Diagram China occurences

,created_at,favorite_count,id_str,in_reply_to_user_id_str,is_retweet,retweet_count,source,text,keyword
12,Mon Jan 02 23:47:12 +0000 2017,64480,816068355555815424,None,False,17507,Twitter for Android,China has been taking out massive amounts of m...,True
233,Sat Feb 04 03:07:47 +0000 2017,154637,827715246135574530,None,False,27046,Twitter for Android,Countries charge U.S. companies taxes or tarif...,True
279,Fri Feb 10 13:35:50 +0000 2017,106953,830047626414477312,None,False,23732,Twitter for Android,The failing @nytimes does major FAKE NEWS Chin...,True
449,Fri Mar 17 13:07:15 +0000 2017,89407,842724011234791424,None,False,20995,Twitter for iPhone,North Korea is behaving very badly. They have ...,True
505,Thu Mar 30 22:16:17 +0000 2017,58102,847573220417044480,None,False,12126,Twitter for iPhone,The meeting next week with China will be a ver...,True
